In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

#### 1.Load the patient dataset and show summary with info().

In [41]:
df_patient = pd.read_csv('datasets/Patient_Data.csv')
df_patient.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


In [ ]:
df_patient.info()

#### 2.Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [ ]:
df_patient[['PatientID','Department','Doctor','BillAmount']]

#### 3.Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [65]:
df_patient.drop(columns=['ReceptionistID','CheckInTime'], inplace=True)

#### 4.Use groupby to find total bill amount per department.

In [31]:
total_bill_amount = df_patient.groupby(['Department'])['BillAmount'].sum()
total_bill_amount

Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64

#### 5.Remove duplicate patient records based on PatientID.

In [37]:
df_patient.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
dtype: bool

In [47]:
df_patient.drop_duplicates(inplace=True)

In [49]:
df_patient

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


#### 6.Fill missing BillAmount values with the mean bill amount.

In [51]:
df_patient.isna().sum()

PatientID         0
Name              0
Department        0
Doctor            0
BillAmount        2
ReceptionistID    0
CheckInTime       0
dtype: int64

In [59]:
df_patient['BillAmount'].fillna(df_patient['BillAmount'].mean(), inplace=True)

#### 7.Merge the billing dataset with patient dataset on PatientID.

In [61]:
df_billing = pd.read_csv('datasets/Billing_Data.csv')
df_billing

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [67]:
data = pd.merge(df_patient, df_billing, on='PatientID')   # inner join
data

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000


#### 8.Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [75]:
current_week_data = {
    'PatientID' : [109],
    'Name' : ['Joe'],
    'Department' : ['Neurology'],
    'Doctor' : ['Dr. John'],
    'BillAmount' : [7800]    
}

current_week = pd.DataFrame(current_week_data)
data = pd.concat([current_week, df_patient], axis=0, ignore_index=True)
data

,PatientID,Name,Department,Doctor,BillAmount
0,109,Joe,Neurology,Dr. John,7800.000000
1,101,Alice,Cardiology,Dr. Smith,5000.000000
2,102,Bob,Neurology,Dr. John,6233.333333
3,103,Charlie,Orthopedics,Dr. Lee,7500.000000
4,104,David,Cardiology,Dr. Smith,6200.000000
5,105,Eva,Dermatology,Dr. Rose,6233.333333


#### 9.Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [79]:
data = pd.concat([df_patient, df_billing['InsuranceCovered'], df_billing['FinalAmount']], axis=1)
data

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000


This is the final bill amount which includes all details about patient's doctor, department, whether insurance covered or not. And the final amount calculated based on the insurance.